In [ ]:
!pip install --upgrade mxnet
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.3/218.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.0 MB/s eta 0:

In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
df = pd.read_csv('superstoretrain.csv')
df.head()

,Order Date,Sales
0,06/01/14,2573.820
1,07/01/14,76.728
2,10/01/14,51.940
3,11/01/14,9.940
4,13/01/14,879.939


In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df = df.sort_values("Order Date")
df

,Order Date,Sales
18,2014-01-03,1893.9950
65,2014-01-06,2024.0600
102,2014-01-08,44.1280
158,2014-01-11,100.3840
180,2014-01-12,4513.6060
...,...,...
472,2016-12-05,10.0200
493,2016-12-06,433.3960
524,2016-12-08,917.2045
546,2016-12-09,1679.8720


In [ ]:
df.set_index(df["Order Date"], inplace=True)

In [ ]:
df = df.resample('D').fillna('ffill')

In [ ]:
df

,Order Date,Sales
Order Date,,
2014-01-03,2014-01-03,1893.9950
2014-01-04,2014-01-03,1893.9950
2014-01-05,2014-01-03,1893.9950
2014-01-06,2014-01-06,2024.0600
2014-01-07,2014-01-06,2024.0600
...,...,...
2016-12-07,2016-12-06,433.3960
2016-12-08,2016-12-08,917.2045
2016-12-09,2016-12-09,1679.8720


In [ ]:
df["item_id"] = 1
df

,Order Date,Sales,item_id
Order Date,,,
2014-01-03,2014-01-03,1893.9950,1
2014-01-04,2014-01-03,1893.9950,1
2014-01-05,2014-01-03,1893.9950,1
2014-01-06,2014-01-06,2024.0600,1
2014-01-07,2014-01-06,2024.0600,1
...,...,...,...
2016-12-07,2016-12-06,433.3960,1
2016-12-08,2016-12-08,917.2045,1
2016-12-09,2016-12-09,1679.8720,1


In [ ]:
df.describe()

,Sales,item_id
count,1074.000000,1074.0
mean,791.799918,1.0
std,972.006183,0.0
min,4.224000,1.0
25%,161.190000,1.0
50%,435.629500,1.0
75%,1108.561000,1.0
max,10006.174000,1.0


In [ ]:
df["Date"] = df.index

In [ ]:
df.set_index(df["item_id"], inplace = True)

In [ ]:
df.drop('Order Date', inplace=True, axis = 1)
df

,Sales,item_id,Date
item_id,,,
1,1893.9950,1,2014-01-03
1,1893.9950,1,2014-01-04
1,1893.9950,1,2014-01-05
1,2024.0600,1,2014-01-06
1,2024.0600,1,2014-01-07
...,...,...,...
1,433.3960,1,2016-12-07
1,917.2045,1,2016-12-08
1,1679.8720,1,2016-12-09


In [ ]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="Date"
)
train_data.head()

Sales
item_id timestamp           
1       2014-01-03  1893.995
        2014-01-04  1893.995
        2014-01-05  1893.995
        2014-01-06  2024.060
        2014-01-07  2024.060

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=30,
    path="autogluon-m4-hourly",
    target="Sales",
    eval_metric="RMSE",
    # ignore_time_index=True
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=600,
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: medium_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'medium_quality',
 'prediction_length': 30,
 'random_seed': None,
 'target': 'Sales',
 'time_limit': 600}
Provided training data set with 1074 rows, 1 items (item = single time series). Average time series length is 1074.0.
Training artifacts will be saved to: /content/autogluon-m4-hourly
AutoGluon will save models to autogluon-m4-hourly/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'Sales'
tuning_data is None. Will use the last prediction_length = 30 time steps of each time series as a hold-out 

In [ ]:
predictions = predictor.predict(train_data)
predictions

INFO:lightning_fabric.utilities.seed:Global seed set to 123
Model not specified in predict, will default to the model with the best validation score: AutoGluonTabular


mean         0.1         0.2          0.3  \
item_id timestamp                                                      
1       2016-12-12   799.226987  300.102983  386.406269   484.726571   
        2016-12-13   782.459376  283.335371  369.638658   467.958959   
        2016-12-14   873.230844  374.106840  460.410126   558.730428   
        2016-12-15   547.661276   48.537272  134.840558   233.160859   
        2016-12-16   570.339584   71.215579  157.518866   255.839167   
        2016-12-17   702.635909  203.511905  289.815191   388.135492   
        2016-12-18   819.063592  319.939588  406.242874   504.563176   
        2016-12-19   846.192478  347.068474  433.371760   531.692061   
        2016-12-20  1172.601739  673.477734  759.781021   858.101275   
        2016-12-21  1314.967391  815.843386  902.146625  1000.466927   
        2016-12-22   878.164888  379.040883  465.344170   563.664471   
        2016-12-23  1030.519446  531.395441  617.698728   716.019029   
        2016-12-24   967.894292  468.770287  555.073574   653.393875   
        2016-12-25   444.249583  -54.874421   31.428865   129.749167   
        2016-12-26   818.135551  319.011546  405.314833   503.635134   
        2016-12-27  1029.675317  530.551312  616.854599   715.174900   
        2016-12-28  1156.080579  656.956575  743.259861   841.580210   
        2016-12-29   749.578086  250.454081  336.757368   435.077669   
        2016-12-30  1006.785953  507.661949  593.965235   692.285537   
        2016-12-31   797.056019  297.932015  384.235301   482.555603   
        2017-01-01  1041.898621  542.774616  629.077903   727.398204   
        2017-01-02   897.794024  398.670019  484.973306   583.293607   
        2017-01-03  1074.914039  575.790034  662.093321   760.413622   
        2017-01-04   955.905358  456.781353  543.084640   641.404941   
        2017-01-05   786.561938  287.437934  373.741220   472.061522   
        2017-01-06   993.111405  493.987400  580.290687   678.610988   
        2017-01-07   782.785587  283.661583  369.964869   468.285170   
        2017-01-08   758.789266  259.665262  345.968548   444.288849   
        2017-01-09   891.261393  392.137388  478.440675   576.760976   
        2017-01-10   988.072211  488.948206  575.251493   673.571794   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
1       2016-12-12   570.481168   670.774172   813.401614  1043.170902   
        2016-12-13   553.713557   654.006560   796.634002  1026.403291   
        2016-12-14   644.485025   744.778029   887.405471  1117.174759   
        2016-12-15   318.915434   419.208460   561.835950   791.605191   
        2016-12-16   341.593741   441.886768   584.514257   814.283499   
        2016-12-17   473.890090   574.183093   716.810583   946.579824   
        2016-12-18   590.317773   690.610777   833.238219  1063.007507   
        2016-12-19   617.446659   717.739663   860.367105  1090.136393   
        2016-12-20   943.855920  1044.148876  1186.776365  1416.545654   
        2016-12-21  1086.221572  1186.514528  1329.142017  1558.911306   
        2016-12-22   649.419069   749.712072   892.339514  1122.108803   
        2016-12-23   801.773627   902.066583  1044.694072  1274.463361   
        2016-12-24   739.148473   839.441523   982.068918  1211.838207   
        2016-12-25   215.503741   315.796768   458.424257   688.193498   
        2016-12-26   589.389732   689.682735   832.310177  1062.079466   
        2016-12-27   800.929498   901.222548  1043.849943  1273.619232   
        2016-12-28   927.334761  1027.627811  1170.255206  1400.024494   
        2016-12-29   520.832267   621.125270   763.752759   993.522001   
        2016-12-30   778.040135   878.333185  1020.960580  1250.729868   
        2016-12-31   568.310201   668.603204   811.230646  1040.999934   
        2017-01-01   813.152802   913.445852  1056.073247  1285.842536   
        2017-01-02  

In [ ]:
predictor.leaderboard(train_data, silent=True)

Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,AutoGluonTabular,-1046.859762,-1046.859762,0.067663,0.119337,10.445405,7
1,ARIMA,-1063.474593,-1063.474593,0.011930,1.081861,0.001032,5
2,AutoETS,-1118.739939,-1118.739939,0.013009,32.083989,0.000889,6
3,DeepAR,-1143.660028,-1157.077789,0.165405,0.200054,555.484167,8
4,Theta,-1206.435371,-1206.435371,0.011926,1.314488,0.000926,4
5,ETS,-1279.705721,-1279.705721,0.012470,0.257980,0.000946,3
6,SeasonalNaive,-1464.184940,-1464.184940,0.012279,0.027915,0.000900,2
7,Naive,-2723.346024,-2723.346024,0.016600,0.046828,0.004521,1


In [ ]:
predictor.leaderboard()

              model    score_val  pred_time_val  fit_time_marginal  fit_order
0  AutoGluonTabular -1046.859762       0.119337          10.445405          7
1             ARIMA -1063.474593       1.081861           0.001032          5
2           AutoETS -1118.739939      32.083989           0.000889          6
3            DeepAR -1157.077789       0.200054         555.484167          8
4             Theta -1206.435371       1.314488           0.000926          4
5               ETS -1279.705721       0.257980           0.000946          3
6     SeasonalNaive -1464.184940       0.027915           0.000900          2
7             Naive -2723.346024       0.046828           0.004521          1


,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,AutoGluonTabular,-1046.859762,0.119337,10.445405,7
1,ARIMA,-1063.474593,1.081861,0.001032,5
2,AutoETS,-1118.739939,32.083989,0.000889,6
3,DeepAR,-1157.077789,0.200054,555.484167,8
4,Theta,-1206.435371,1.314488,0.000926,4
5,ETS,-1279.705721,0.257980,0.000946,3
6,SeasonalNaive,-1464.184940,0.027915,0.000900,2
7,Naive,-2723.346024,0.046828,0.004521,1


In [ ]:
predictor.fit_summary()

****************** Summary of fit() ******************
Estimated performance of each model:
              model    score_val  pred_time_val  fit_time_marginal  fit_order
0  AutoGluonTabular -1046.859762       0.119337          10.445405          7
1             ARIMA -1063.474593       1.081861           0.001032          5
2           AutoETS -1118.739939      32.083989           0.000889          6
3            DeepAR -1157.077789       0.200054         555.484167          8
4             Theta -1206.435371       1.314488           0.000926          4
5               ETS -1279.705721       0.257980           0.000946          3
6     SeasonalNaive -1464.184940       0.027915           0.000900          2
7             Naive -2723.346024       0.046828           0.004521          1
Number of models trained: 8
Types of models trained:
{'ThetaModel', 'SeasonalNaiveModel', 'AutoGluonTabularModel', 'NaiveModel', 'ETSModel', 'AutoETSModel', 'ARIMAModel', 'DeepARModel'}
****************** E

{'model_types': {'Naive': 'NaiveModel',
  'SeasonalNaive': 'SeasonalNaiveModel',
  'ETS': 'ETSModel',
  'Theta': 'ThetaModel',
  'ARIMA': 'ARIMAModel',
  'AutoETS': 'AutoETSModel',
  'AutoGluonTabular': 'AutoGluonTabularModel',
  'DeepAR': 'DeepARModel'},
 'model_performance': {'Naive': -2723.3460239630417,
  'SeasonalNaive': -1464.1849402089122,
  'ETS': -1279.7057209599202,
  'Theta': -1206.4353705714188,
  'ARIMA': -1063.4745925948764,
  'AutoETS': -1118.739938703811,
  'AutoGluonTabular': -1046.8597623751168,
  'DeepAR': -1157.0777888961095},
 'model_best': 'AutoGluonTabular',
 'model_paths': {'Naive': 'autogluon-m4-hourly/models/Naive/',
  'SeasonalNaive': 'autogluon-m4-hourly/models/SeasonalNaive/',
  'ETS': 'autogluon-m4-hourly/models/ETS/',
  'Theta': 'autogluon-m4-hourly/models/Theta/',
  'ARIMA': 'autogluon-m4-hourly/models/ARIMA/',
  'AutoETS': 'autogluon-m4-hourly/models/AutoETS/',
  'AutoGluonTabular': 'autogluon-m4-hourly/models/AutoGluonTabular/',
  'DeepAR': 'autogluon-

In [ ]:
!zip -r /content/file.zip /content/autogluon-m4-hourly

  adding: content/autogluon-m4-hourly/ (stored 0%)
  adding: content/autogluon-m4-hourly/utils/ (stored 0%)
  adding: content/autogluon-m4-hourly/utils/data/ (stored 0%)
  adding: content/autogluon-m4-hourly/utils/data/val.pkl (deflated 47%)
  adding: content/autogluon-m4-hourly/utils/data/train.pkl (deflated 47%)
  adding: content/autogluon-m4-hourly/learner.pkl (deflated 55%)
  adding: content/autogluon-m4-hourly/predictor.pkl (deflated 30%)
  adding: content/autogluon-m4-hourly/models/ (stored 0%)
  adding: content/autogluon-m4-hourly/models/trainer.pkl (deflated 53%)
  adding: content/autogluon-m4-hourly/models/ETS/ (stored 0%)
  adding: content/autogluon-m4-hourly/models/ETS/model.pkl (deflated 28%)
  adding: content/autogluon-m4-hourly/models/AutoGluonTabular/ (stored 0%)
  adding: content/autogluon-m4-hourly/models/AutoGluonTabular/metadata.json (deflated 67%)
  adding: content/autogluon-m4-hourly/models/AutoGluonTabular/utils/ (stored 0%)
  adding: content/autogluon-m4-hourly/m

In [ ]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# train_data.tail()
len(predictions)

30

In [ ]:
import matplotlib.pyplot as plt


df2 = pd.read_csv('superstoretest.csv')

df2['timestamp'] = pd.to_datetime(df2['Order Date'])
# df2.set_index(df['Order Date'], inplace=True)

df2

,item_id,Order Date,Sales,timestamp
0,1,12/12/16,477.4700,2016-12-12
1,1,13/12/16,1114.2720,2016-12-13
2,1,14/12/16,611.7240,2016-12-14
3,1,15/12/16,2231.8950,2016-12-15
4,1,16/12/16,563.9200,2016-12-16
...,...,...,...,...
270,1,24/12/17,1393.4940,2017-12-24
271,1,25/12/17,832.4540,2017-12-25
272,1,28/12/17,551.2568,2017-12-28
273,1,29/12/17,2330.7180,2017-12-29


In [ ]:
df2.set_index(df2['timestamp'], inplace=True)

In [ ]:
df2

,item_id,Order Date,Sales,timestamp
timestamp,,,,
2016-12-12,1,12/12/16,477.4700,2016-12-12
2016-12-13,1,13/12/16,1114.2720,2016-12-13
2016-12-14,1,14/12/16,611.7240,2016-12-14
2016-12-15,1,15/12/16,2231.8950,2016-12-15
2016-12-16,1,16/12/16,563.9200,2016-12-16
...,...,...,...,...
2017-12-24,1,24/12/17,1393.4940,2017-12-24
2017-12-25,1,25/12/17,832.4540,2017-12-25
2017-12-28,1,28/12/17,551.2568,2017-12-28


In [ ]:
df.drop('Order Date', inplace = True, axis = 1)

In [ ]:
# df2.set_index(df['timestamp'], inplace=True)

In [ ]:
df2 = df2.resample('D').fillna('ffill')

In [ ]:
df2.set_index(df["item_id"], inplace = True)

ValueError: ignored

In [ ]:
df2.drop("item_id", axis = 1)

In [ ]:
predictions.index = pd.to_datetime(predictions.index)

In [ ]:
y_past.head(10)

In [ ]:
predictions

In [ ]:
y_test

In [ ]:

# TimeSeriesDataFrame can also be loaded directly from a file
test_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp"
)
plt.figure(figsize=(20, 3))

item_id = 1
y_past = train_data.loc[item_id]["Sales"]
y_pred = predictions.loc[item_id]
y_test = test_data.loc[item_id]["Sales"][:70]

plt.plot(y_past[-200:], label="Past time series values")
plt.plot(y_pred["0.9"], label="Mean forecast")
plt.plot(y_test, label="Future time series values")

# plt.fill_between(
#     y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
# )
plt.legend();

In [ ]:
predictions.to_csv

In [ ]:
predictor.leaderboard(test_data, silent=True)